# twitter agent
beta

## Install Python Libraries

- openai-agents

In [10]:
pip install openai-agents openai pymongo tweepy -q

Note: you may need to restart the kernel to use updated packages.


## Purchase and Store API Key

Create a MongoDB cluster and store the connection string in a safe place, such as AWS Secrets Manager.

    key name: connection_string
    key value: <the connection string>, you need to type the password
    secret name: mongodb


You need to **purchase** your [OpenAI](https://openai.com/) API key and store it securely, such as in **AWS Secrets Manager**.

- **Key Name:** `api_key`  
- **Key Value:** `<your OpenAI API key>`  
- **Secret Name:** `openai`


## Import Required Libraries

The following libraries are used in this notebook:

- **boto3**: AWS SDK for Python, used to interact with AWS services.
- **json**: Standard Python library for handling JSON data.
- **IPython.display**: Provides tools to display images, Markdown content, and other rich media in Jupyter Notebook.
- **pandas**: A powerful library for data manipulation and analysis.
- **pprint**: Pretty prints data structures for better readability.

In [22]:
import boto3
import json
from IPython.display import display, Image, Markdown
import pandas as pd
from pprint import pprint
import pymongo
from pymongo import MongoClient
from agents import Agent, Runner, set_default_openai_key


# import tools

In [30]:
import sys
import os

print(os.listdir())
from twitter_analysis_tools import collect_tweets

['Analyze_Tweets_Responses_API.ipynb', 'Prompt-Engineering-Analyze-Twitter-Data.ipynb', 'Automating-Grading-with-OpenAI-Function-Calling.ipynb', 'OpenAI-Assistant-Crafting-Email-Responses.ipynb', 'Collect_Twitter_Data.ipynb', 'Collect-Job-Data-with-Generative-AI.ipynb', '.gitignore', 'Twitter-Image-Classification-Recreation-Editing.ipynb', 'Twitter_Data_Agent', 'LICENSE', 'Exploring-Twitter-Data-with-Vector-Databases-and-RAG-Systems.ipynb', 'Openai_o1_Reasoning_vs_GPT.ipynb', '.ipynb_checkpoints', 'README.md', '.git', 'diamonds.csv']


ModuleNotFoundError: No module named 'twitter_analysis_tools'

## Retrieve API Keys Securely from AWS Secrets Manager

The following function, `get_secret()`, retrieves a secret from **AWS Secrets Manager**. This is a secure way to store and access sensitive credentials, such as API keys, without hardcoding them into the script

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

configu api key

In [17]:
bearer_token   = get_secret('twitter_api')['bearer_token']

mongodb_connect = get_secret('mongodb')['connection_string']

set_default_openai_key(get_secret('openai')['api_key'])



## Connect to the MongoDB cluster

In [15]:
mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

# Define the Twitter Data Collector Agent

In [19]:
data_collector_agent = Agent(
    name="DataCollectorAgent",
    instructions=(
        "Collect tweets based on a topic and number. "
        "Use the 'collect_tweets' tool to save them to MongoDB."
    ),
    tools=[collect_tweets],
)


NameError: name 'collect_tweets' is not defined

# Define the Main Orchestrator Agent

In [20]:
main_agent = Agent(
    name="MainTwitterAgent",
    instructions=(
        "You are a coordinator agent. Based on the user's request, decide whether to collect tweets, analyze tweets, or answer questions. "
        "Start by using the appropriate agent to complete the task."
    ),
    tools=[data_collector_agent],
)

NameError: name 'data_collector_agent' is not defined

# Example Usage: Direct user query

In [21]:
response = Runner.run_sync(main_agent, "Collect 50 tweets about generative AI")
print(response.final_output)

NameError: name 'main_agent' is not defined